In [0]:
import numpy as np
from tensorflow.keras import preprocessing , utils
import string
import tensorflow as tf
# NLTK
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [0]:
#  Loading Data
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('qa_Electronics.json.gz')

In [0]:
pd.set_option('display.max_columns',None)

In [0]:
df.head(10)

,questionType,asin,answerTime,unixTime,question,answerType,answer
0,yes/no,0594033926,"Dec 27, 2013",1.388131e+09,Is this cover the one that fits the old nook c...,Y,Yes this fits both the nook color and the same...
1,yes/no,0594033926,"Jan 5, 2015",1.420445e+09,Does it fit Nook GlowLight?,N,No. The nook color or color tablet
2,open-ended,0594033926,2 days ago,NaN,Would it fit Nook 1st Edition? 4.9in x 7.7in ?,NaN,I don't think so. The nook color is 5 x 8 so n...
3,yes/no,0594033926,17 days ago,NaN,Will this fit a Nook Color that's 5 x 8?,Y,yes
4,yes/no,0594033926,"Feb 10, 2015",1.423555e+09,will this fit the Samsung Galaxy Tab 4 Nook 10.1,N,"No, the tab is smaller than the 'color'"
5,yes/no,0594033926,"Jan 30, 2015",1.422605e+09,does it have a flip stand?,N,"No, there is not a flip stand. It has a pocket..."
6,yes/no,0594033926,"Jan 30, 2015",1.422605e+09,does this have a flip stand,?,"Hi, no it doesn't"
7,open-ended,0594033926,"Dec 22, 2014",1.419235e+09,also fits the HD+?,NaN,It should. They are the same size and the char...
8,yes/no,0594033926,"Nov 16, 2014",1.416125e+09,Does it have 2 positions for the reader? Horiz...,Y,Yes
9,open-ended,0594033926,"Aug 7, 2014",1.407395e+09,"Is there a closure mechanism? Bands, magnetic,...",NaN,No- it is more like a normal book would be. It...


In [0]:
null_id = []
for i, val in enumerate(df['question']):
    if len(val) <= 10:
        null_id.append(i)

In [0]:
df =  df.drop(df.index[null_id])

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 312129 entries, 0 to 314262
Data columns (total 7 columns):
questionType    312129 non-null object
asin            312129 non-null object
answerTime      312129 non-null object
unixTime        302749 non-null float64
question        312129 non-null object
answerType      165529 non-null object
answer          312129 non-null object
dtypes: float64(1), object(6)
memory usage: 19.1+ MB


In [0]:
df['asin'].nunique()

39371

In [0]:
df1 = df.drop(['questionType','asin','answerTime','unixTime','answerType'], axis =1)

In [0]:
df1.head()

,question,answer
0,Is this cover the one that fits the old nook c...,Yes this fits both the nook color and the same...
1,Does it fit Nook GlowLight?,No. The nook color or color tablet
2,Would it fit Nook 1st Edition? 4.9in x 7.7in ?,I don't think so. The nook color is 5 x 8 so n...
3,Will this fit a Nook Color that's 5 x 8?,yes
4,will this fit the Samsung Galaxy Tab 4 Nook 10.1,"No, the tab is smaller than the 'color'"


#### Convert text to lowercase

In [0]:
df2 = df1.apply(lambda x: x.astype(str).str.lower())

In [0]:
df2.head(10)

,question,answer
0,is this cover the one that fits the old nook c...,yes this fits both the nook color and the same...
1,does it fit nook glowlight?,no. the nook color or color tablet
2,would it fit nook 1st edition? 4.9in x 7.7in ?,i don't think so. the nook color is 5 x 8 so n...
3,will this fit a nook color that's 5 x 8?,yes
4,will this fit the samsung galaxy tab 4 nook 10.1,"no, the tab is smaller than the 'color'"
5,does it have a flip stand?,"no, there is not a flip stand. it has a pocket..."
6,does this have a flip stand,"hi, no it doesn't"
7,also fits the hd+?,it should. they are the same size and the char...
8,does it have 2 positions for the reader? horiz...,yes
9,"is there a closure mechanism? bands, magnetic,...",no- it is more like a normal book would be. it...


In [0]:
questions =  df2['question'].values.tolist()
questions = questions[0:len(questions)//312]
print(len(questions))
questions[:10]

1000


['is this cover the one that fits the old nook color? which i believe is 8x5.',
 'does it fit nook glowlight?',
 'would it fit nook 1st edition? 4.9in x 7.7in ?',
 "will this fit a nook color that's 5 x 8?",
 'will this fit the samsung galaxy tab 4 nook 10.1',
 'does it have a flip stand?',
 'does this have a flip stand',
 'also fits the hd+?',
 'does it have 2 positions for the reader? horizontal/vertical thank you kwod',
 'is there a closure mechanism? bands, magnetic, etc.?']

3121

In [0]:
answers = df2['answer'].values.tolist()
answers = answers[0:len(answers)//312]
print(len(answers))
answers[:10]

1000


['yes this fits both the nook color and the same-shaped nook tablet',
 'no. the nook color or color tablet',
 "i don't think so. the nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close.",
 'yes',
 "no, the tab is smaller than the 'color'",
 'no, there is not a flip stand. it has a pocket in the front flap. it is a very nice cover.',
 "hi, no it doesn't",
 'it should. they are the same size and the charging port is in the same place.',
 'yes',
 "no- it is more like a normal book would be. it doesn't flop open- so it has never wen an issue for me. the nook clips into a secure and safe holder inside that is small and convenient. this is the best cover i've ever had for my nook- trim and functional (no magnets or elastic needed)."]

In [0]:
answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 5154


In [0]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

(1000, 52) 52


In [0]:
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

(1000, 612) 612


In [0]:
# decoder_output_data -1 
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )

In [0]:
padded_answers

array([[ 33,  10, 436, ...,   0,   0,   0],
       [ 36,   1,  48, ...,   0,   0,   0],
       [  5,  58, 169, ...,   0,   0,   0],
       ...,
       [  1, 281,  43, ...,   0,   0,   0],
       [  5,  58, 169, ...,   0,   0,   0],
       [ 33,   3, 186, ...,   0,   0,   0]])

In [0]:
# # decoder_output_data -2

onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
# onehot_answers = sparse_categorical_crossentropy( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

(1000, 612, 5154)


In [0]:
# Saving all the arrays to storage
np.save( 'encoder_input_data.npy' , encoder_input_data )
np.save( 'decoder_input_data.npy' , decoder_input_data )
np.save( 'decoder_output_data.npy' , decoder_output_data )

### Model Training

In [0]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

In [0]:
decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 200)    1030800     input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 200)    1030800     input_4[0][0]                    
____________________________________________________________________________________________

In [0]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=10, epochs=50, use_multiprocessing= True) 
# model.save( 'model.h5' )

Train on 1000 samples
Epoch 1/50
1000/1000 [==============================] - 247s 247ms/sample - loss: 0.4015
Epoch 2/50
1000/1000 [==============================] - 201s 201ms/sample - loss: 0.3826
Epoch 3/50
1000/1000 [==============================] - 163s 163ms/sample - loss: 0.3738
Epoch 4/50
1000/1000 [==============================] - 154s 154ms/sample - loss: 0.3600
Epoch 5/50
1000/1000 [==============================] - 145s 145ms/sample - loss: 0.3502
Epoch 6/50
1000/1000 [==============================] - 144s 144ms/sample - loss: 0.3418
Epoch 7/50
1000/1000 [==============================] - 145s 145ms/sample - loss: 0.3344
Epoch 8/50
1000/1000 [==============================] - 144s 144ms/sample - loss: 0.3275
Epoch 9/50
1000/1000 [==============================] - 145s 145ms/sample - loss: 0.3212
Epoch 10/50
1000/1000 [==============================] - 147s 147ms/sample - loss: 0.3151
Epoch 11/50
1000/1000 [==============================] - 147s 147ms/sample - loss: 0.30

In [0]:
model.save( 'model_lstm.h5' )

In [0]:
pkl_filename = "model_lstm_pkl2.pkl"

saved_model = pickle.dumps(pkl_filename) 

In [0]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [0]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [0]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

Enter question : will this fit a nook color that's 5 x 8
 yes end
Enter question : how are you
 this is a standard usb cable for a be version for a usb version jack end
Enter question : size
 hi of the screen states flap compatible with the film i hope this helps end
Enter question : does it fit nook glowlight
 no the nook color or color tablet i have a 2 2 inch and it works great a great charger but you can use it for it for this is the best to have a few 8 end
